# SK RAG Pattern Foundational Concepts

Learning objectives:

- RAG pattern foundational concepts (igention, chucking, grounding, retrieval, LLM calling)
- SK Memories with RAM or Sqlite as VectorDB

Possible Memory Stores:
- RAM
- SQLite
- PGVector

## Setup

### Load required .NET packages and supporting constants, classes, etc.

In [ ]:
#r "nuget: dotenv.net"
#r "nuget: Microsoft.SemanticKernel, 1.47.0"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.47.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.47.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.Sqlite, 1.47.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.Postgres, 1.47.0-alpha"
#r "nuget: Npgsql"

#r "nuget: dotenv.net"

using System;

using System.IO;
using System.Net.Http;
using System.Text;
using System.Text.RegularExpressions;
using System.Text.Json;
using System.Text.Json.Serialization;
using Npgsql;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.AzureOpenAI;
using Microsoft.SemanticKernel.Connectors.Sqlite;
using Microsoft.SemanticKernel.Connectors.Postgres;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Plugins.Memory;

using dotenv.net;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

#!import Models/Models.cs

const string MemoryCollectionName = "LearningsCollection";

#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003
#pragma warning disable SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025
#pragma warning disable SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,
#pragma warning disable SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102


### Read the API Key and endpoints from environment variables or the .env file

In [ ]:
// Load the .env file
DotEnv.Load();

// Get the OpenAI deployment name, endpoint, and key from the environment variables
var deploymentName = Environment.GetEnvironmentVariable("GPT_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("GPT_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
var pg_conn_str = Environment.GetEnvironmentVariable("PG_CONN_STR");
var adaDeploymentName = Environment.GetEnvironmentVariable("GPT_EMBEDDING_MODEL");

### Configure the Kernel and memory

In [ ]:
// Note: Added this because I am having problems with SSL certificate validation
var handler = new HttpClientHandler();
handler.CheckCertificateRevocationList = false;
var httpClient = new HttpClient(handler);

In [ ]:
#pragma warning disable SKEXP0010
var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: deploymentName,
        endpoint: endpoint,
        apiKey: apiKey,
        httpClient: httpClient)
    .AddAzureOpenAITextEmbeddingGeneration(adaDeploymentName, endpoint, apiKey, httpClient: httpClient)
    .Build();

In [ ]:
#pragma warning disable SKEXP0050,SKEXP0010,SKEXP0001
var ramStore = new Microsoft.SemanticKernel.Memory.VolatileMemoryStore();

var embeddingGenerator = new AzureOpenAITextEmbeddingGenerationService(
            deploymentName: adaDeploymentName,
            endpoint: endpoint,
            apiKey: apiKey);

var memory = new MemoryBuilder()
            .WithTextEmbeddingGeneration(embeddingGenerator)
            .WithMemoryStore(new Microsoft.SemanticKernel.Memory.VolatileMemoryStore())
            .Build();

### Configure the memory store

One of the amazing features of SK is that, for the memory store object it implements an interface. This means that you can quickly swap memory stores like sqlite, Redis, PostgreSQL, etc. or create a new store as long as it implements the interface.

Below you can see different stores. This notebook is using VolatileMemory (RAM)
```c#
//var memoryStore = await Microsoft.SemanticKernel.Connectors.Postgres.PostgresMemoryStore("Host=localhost;Port=5432;Username=postgres;Password=postgres;Database=postgres");
//var memoryStore = await SqliteMemoryStore.ConnectAsync("./vectors.sqlite");
//NpgsqlDataSourceBuilder dataSourceBuilder = new(pg_conn_str);
//dataSourceBuilder.UseVector();
//NpgsqlDataSource dataSource = dataSourceBuilder.Build();
//IMemoryStore memoryStore = new PostgresMemoryStore(dataSource, vectorSize: 1536, schema: "public");
```

In [ ]:
// Using VolatileMemory, but I can switch providers like Azure Search, DuckDB, SQLite, etc.
#pragma warning disable SKEXP0050
var memoryStore = new Microsoft.SemanticKernel.Memory.VolatileMemoryStore();
memoryStore

### Configure the text memory

In [ ]:
#pragma warning disable SKEXP0001
Microsoft.SemanticKernel.Memory.SemanticTextMemory textMemory = new(memoryStore, embeddingGenerator);
textMemory

## Ingestion

### Read and deserialize the JSON learnings data file

In [ ]:
var jsonFileContents = File.ReadAllText("data/learnings.json");
var learnings = System.Text.Json.JsonSerializer.Deserialize<List<Learning>>(jsonFileContents);
learnings

### Chunk the learnings & recommendations

**Note:** This is a simple chunker. It chunks by splitting the document into paragraphs. A more realistic chunker would try to optimize the token size limit, chunking smartly (not in the a middle of a paragraph or sentence), etc.

In [ ]:
// Keep a list of chunks
var chunks = new List<Chunk>();

// For each learning process the chunks
foreach(var learning in learnings)
{
    // Break the learnings into paragraphs
    var paragraphs = learning.Content.Split("\n\n");
    
    // For each paragraph create a chunk
    for(var i=0;i<paragraphs.Length;i++)
    {
        // Add the chunk to the list
        chunks.Add(new Chunk(learning.Id+"-"+(i+1),paragraphs[i],"data/learnings.json"));
    }
}
chunks

### Save every chunk as a memory

In [ ]:
// Create an embedding generator to use for semantic memory.
foreach(var chunk in chunks)
{    
    await textMemory.SaveInformationAsync(MemoryCollectionName, id: chunk.Id, text: chunk.Text);
}

## Grounding

### Retrieve the memory based on a query

In [ ]:
//var query = await InteractiveKernel.GetInputAsync("What is your query?");
var question = "What scenario is FrontDoor good for?";

#pragma warning disable SKEXP0001
IAsyncEnumerable<MemoryQueryResult> queryResults =
                textMemory.SearchAsync(MemoryCollectionName, question, limit: 3, minRelevanceScore: 0.1);


### Find memories based on query, and collect the text in the memories to augment the prompt

In [ ]:
// Keep the text for the recalled memories
StringBuilder memoryText = new StringBuilder();

#pragma warning disable SKEXP0001
await foreach (MemoryQueryResult r in queryResults)
{
    // Append the text
    memoryText.Append(r.Metadata.Text+"\n\n");
}

// Final augmented text
var promptContext = memoryText.ToString();
Console.WriteLine($"User:\n{question}\n\nNearest results:\n{promptContext}")

## Process Prompt & Completion

### Create a SK function

In [ ]:
const string promptTemplate = "{{$input}}\n\nText:\n\"\"\"{{$context}}\n\"\"\"Use only the provided text.";
var skPromptTemplateFunction = kernel.CreateFunctionFromPrompt(promptTemplate);

### Submit the prompt with the orignal questions, retrieved chunks, and print the results

In [ ]:
var arguments = new KernelArguments()
        {
            ["input"] = question,
            ["context"] = promptContext
        };
var result = await kernel.InvokeAsync(skPromptTemplateFunction, arguments);

Console.WriteLine(result);